In [3]:
#import libraries
from functools import reduce
import pandas as pd
import numpy as np
import warnings
import random
import re
import string
from tqdm import tqdm
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import time
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
#Load and view the dataset
df = pd.read_csv('datasets/entity_pairs.csv')
df.head()

,entity
0,Dr. Teresa Y Hughes 52755 Hinton Course Aurora...
1,Ms. Amanda C Davis 4809 Colin Plaza Apt. 412 S...
2,Dr. Kimberly V Anderson 303 Butler Vista Orlan...
3,Dr. Mary H Blair 478 Aaron Valley Syracuse NY ...
4,Kristina Y Baker 4092 Steven Villages New York...


In [5]:
#Create a list of all the entities
entities = []
for i in range(0, df.shape[0]):
    entity_dict = {}
    entity_dict['attributes'] = df['entity'].values[i]
    entity_dict['entity_id'] = df['ID'].values[i]
    entities.append(entity_dict)

In [6]:
#initialize a qdrant vectorDB and sentence transformer to create embeddings
encoder = SentenceTransformer('all-MiniLM-L6-v2', trust_remote_code=True)
client = QdrantClient(host = 'localhost', port = 6333)
client.recreate_collection(
    collection_name="entities",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

#Add entities to vectorDB
client.upload_points(
    collection_name="entities",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["attributes"]).tolist(), payload=doc
        )
        for idx, doc in tqdm(enumerate(entities))
    ],
)

141696it [20:58, 112.62it/s]


In [ ]:
print(random.choice())